In [ ]:
# To avoid crawling the same page twice, it is extremely important that all 
# internal links discovered are formatted consistently, and kept in a running set 
# for easy lookups, while the program is running. 

# Only links that are “new” should be crawled and searched for additional links.

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
pages = set() #Initially empty
def getLinks(pageUrl):
    global pages
    html = urlopen('http://en.wikipedia.org{}'.format(pageUrl)) #Front page of wikipedia
    bs = BeautifulSoup(html, 'html.parser')
    for link in bs.find_all('a', href=re.compile('^(/wiki/)')): #Each link on first page is iterated
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages: #Is it the page already in global pages?
                #We have encountered a new page
                newPage = link.attrs['href']
                print(newPage)
                pages.add(newPage) #Add the new page to the pages
                getLinks(newPage) #Called recursively
getLinks('')

# A Warning Regarding Recursion: if left running long enough, the preceding program will almost certainly crash.
# Python has a default recursion limit (the number of times a program can recursively call itself) of 1,000. 

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
pages = set() #Initially empty
def getLinks(pageUrl):
    global pages
    html = urlopen('http://en.wikipedia.org{}'.format(pageUrl)) #Front page of wikipedia
    bs = BeautifulSoup(html, 'html.parser')
    try:
        print(bs.h1.get_text()) #Printing h1 title from the page
        print(bs.find(id ='mw-content-text').find_all('p')[0]) #Printing the 1st <p> within div#mw-content-text 
        print(bs.find(id='ca-edit').find('span')
             .find('a').attrs['href']) 
    except AttributeError:
        print('This page is missing something! Continuing.')

    for link in bs.find_all('a', href=re.compile('^(/wiki/)')): #Each link on first page is iterated
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages: #Is it the page already in global pages?
                #We have encountered a new page
                newPage = link.attrs['href']
                print('-'*20)
                print(newPage)
                pages.add(newPage) #Add the new page to the pages
                getLinks(newPage) #Called recursively
getLinks('')

# you can never be entirely sure that all the data is on each page, each print
# statement is arranged in the order that it is likeliest to appear on the site.

In [ ]:
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import random
pages = set()
seed = None
random.seed(seed)

#Retrieves a list of all Internal links found on a page
def getInternalLinks(bs, includeUrl):
    includeUrl = '{}://{}'.format(urlparse(includeUrl).scheme, urlparse(includeUrl).netloc)
    internalLinks = []
    #Finds all links that begin with a "/"
    for link in bs.find_all('a', href=re.compile('^(/|.*'+includeUrl+')')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
                if(link.attrs['href'].startswith('/')):
                    internalLinks.append(includeUrl+link.attrs['href'])
                else:
                    internalLinks.append(link.attrs['href'])
    return internalLinks

#Retrieves a list of all external links found on a page
def getExternalLinks(bs, excludeUrl):
    externalLinks = []
    #Finds all links that start with "http" that do not contain the current URL
    for link in bs.find_all('a', href=re.compile('^(http|www)((?!'+excludeUrl+').)*$')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks

def getRandomExternalLink(startingPage):
    html = urlopen(startingPage)
    bs = BeautifulSoup(html, 'html.parser')
    externalLinks = getExternalLinks(bs, urlparse(startingPage).netloc)
    if len(externalLinks) == 0:
        print('No external links, looking around the site for one')
        domain = '{}://{}'.format(urlparse(startingPage).scheme, urlparse(startingPage).netloc)
        internalLinks = getInternalLinks(bs, domain)
        return getRandomExternalLink(internalLinks[random.randint(0, len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0, len(externalLinks)-1)]

def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink(startingSite)
    print('Random external link is: {}'.format(externalLink))
    followExternalOnly(externalLink)
    followExternalOnly('https://oreilly.com')

In [ ]:
# Collects a list of all external URLs found on the site
allExtLinks = set()
allIntLinks = set()

def getAllExternalLinks(siteUrl):
    html = urlopen(siteUrl)
    domain = '{}://{}'.format(urlparse(siteUrl).scheme, urlparse(siteUrl).netloc)
    bs = BeautifulSoup(html, 'html.parser')
    internalLinks = getInternalLinks(bs, domain)
    externalLinks = getExternalLinks(bs, domain)

    for link in externalLinks:
        if link not in allExtLinks:
            allExtLinks.add(link)
            print(link)
    for link in internalLinks:
        if link not in allIntLinks:
            allIntLinks.add(link)
            getAllExternalLinks(link)
            
allIntLinks.add('http://oreilly.com')
getAllExternalLinks('http://oreilly.com')
